In [83]:
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd
import os
import math

In [84]:
mp_face_mes = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mes.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [85]:
images_open = os.listdir('eye_open') #Warning: in all images, BOTH eyes must be open
images_closed = os.listdir('eye_closed') #Warning: in all images, BOTH eyes must be closed

print(len(images_open), len(images_closed))

88 60


In [86]:
def calculate_roll_angle(landmarks):
    left_ear_x = landmarks[234].x
    left_ear_y = landmarks[234].y
    right_ear_x = landmarks[454].x
    right_ear_y = landmarks[454].y
    angle_rad = math.atan2(right_ear_y - left_ear_y, right_ear_x - left_ear_x)
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def compensate_head_roll(frame, results_mesh, W, H):
    roll_angle_deg = calculate_roll_angle(results_mesh.multi_face_landmarks[0].landmark)
    M = cv2.getRotationMatrix2D((W/2, H/2), roll_angle_deg, 1)
    frame = cv2.warpAffine(frame, M, (W, H))
    return frame

In [87]:
def update_mesh_points(image):
    try:
        frame = image.copy()
    except AttributeError:
        return None
    H, W, _ = frame.shape
    results_mesh_local = face_mesh.process(frame)
    if results_mesh_local.multi_face_landmarks:
        frame = compensate_head_roll(frame, results_mesh_local, W, H)
        results_mesh = face_mesh.process(frame)
        if results_mesh.multi_face_landmarks:
            mesh_points = np.array([np.multiply([p.x, p.y], [W, H]).astype(int) for p in results_mesh.multi_face_landmarks[0].landmark])
            for point in mesh_points:
                cv2.circle(frame, tuple(point), 1, (0, 255, 0), -1)
            return mesh_points
    return None

In [88]:
RIGHT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
LEFT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

def get_width_over_height(frame):
    mesh_points = update_mesh_points(frame)
    if mesh_points is not None:
        lex1, ley1 = np.min(mesh_points[LEFT_EYE], axis=0)
        lex2, ley2 = np.max(mesh_points[LEFT_EYE], axis=0)
        rex1, rey1 = np.min(mesh_points[RIGHT_EYE], axis=0)
        rex2, rey2 = np.max(mesh_points[RIGHT_EYE], axis=0)
        reason_left = abs(lex1-lex2)/abs(ley1-ley2)
        reason_right = abs(rex1-rex2)/abs(rey1-rey2)
        return (reason_left, reason_right)
    return None

In [89]:
results_open = []
results_closed = []

for image in images_open:
    frame = cv2.imread('eye_open/'+image)
    result = get_width_over_height(frame)
    if result:
        if result[0] > 4 or result[1] > 4:
            print('eye_open/'+image)
        else:
            results_open.append(result[0])
            results_open.append(result[1])

for image in images_closed:
    frame = cv2.imread('eye_closed/'+image)
    result = get_width_over_height(frame)
    if result:
        if result[0] < 6 or result[1] < 6:
            print('eye_closed/'+image)
        else:
            results_closed.append(result[0])
            results_closed.append(result[1])

eye_closed/12058.jpg
eye_closed/360_F_276371400_t9dtc2OOXwcq7mEY1JTuNxdT06nf63Np.jpg


C:\Users\felip\AppData\Local\Temp\ipykernel_18620\2989747734.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  reason_right = abs(rex1-rex2)/abs(rey1-rey2)


eye_closed/7911.jpg
eye_closed/beautiful-girl-with-closed-eyes-touching-her-neck-over-white-background_HKxwxgCrj_SB_PM.jpg
eye_closed/black-woman-eyes-closed-front-view-46737479.jpg
eye_closed/facebook-ai-closed-eyes-photo-fix-open.jpg
eye_closed/portrait-of-real-caucasian-man-with-blank-expression-and-eyes-closed.jpg
eye_closed/preview16 (1).jpg


In [90]:
print(len(results_open), len(results_closed))

130 66


In [91]:
results_open = np.array(results_open)
results_closed = np.array(results_closed)

In [92]:
df = pd.DataFrame(columns=['width_over_height', 'label'])
for value in results_open:
    if value < 1 or value > 15:
        continue
    df = df.append({'width_over_height': value, 'label': 0}, ignore_index=True)
for value in results_closed:
    if value < 1 or value > 15:
        continue
    df = df.append({'width_over_height': value, 'label': 1}, ignore_index=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   width_over_height  194 non-null    float64
 1   label              194 non-null    float64
dtypes: float64(2)
memory usage: 3.2 KB


C:\Users\felip\AppData\Local\Temp\ipykernel_18620\1239552496.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'width_over_height': value, 'label': 0}, ignore_index=True)
C:\Users\felip\AppData\Local\Temp\ipykernel_18620\1239552496.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'width_over_height': value, 'label': 0}, ignore_index=True)
C:\Users\felip\AppData\Local\Temp\ipykernel_18620\1239552496.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'width_over_height': value, 'label': 0}, ignore_index=True)
C:\Users\felip\AppData\Local\Temp\ipykernel_18620\1239552496.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

In [93]:
df

,width_over_height,label
0,2.000000,0.0
1,2.000000,0.0
2,3.500000,0.0
3,2.333333,0.0
4,2.333333,0.0
...,...,...
189,6.625000,1.0
190,6.250000,1.0
191,6.571429,1.0
192,6.500000,1.0


In [94]:
df.to_csv('data.csv', index=False)